In [83]:
import tensorflow as tf
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg.eigen.arpack import eigsh
import scipy.sparse as sp
import pandas as pd
import pickle
from datetime import timedelta
import matplotlib.pyplot as plt
from scipy.stats.stats import pearsonr
import datetime

# Model Architecture

In [84]:
def normalize_adj(adj):
    """Symmetrically normalize adjacency matrix."""
    #adj[np.isnan(adj)] = 0.
    adj = tf.abs(adj)
    rowsum = tf.reduce_sum(adj, 1)# sum by row

    d_inv_sqrt = tf.pow(rowsum, -0.5)
   
    #d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    
    d_mat_inv_sqrt = tf.diag(d_inv_sqrt)

    return tf.matmul(tf.matmul(d_mat_inv_sqrt, adj), d_mat_inv_sqrt)

In [93]:
# Create model
def gcn(x, weights, biases, batch_size, flag, n_output_vec, num = None):
    # Hidden layer with RELU activation
    
    output_list = tf.Variable(tf.zeros([n_output_vec,1]),dtype=tf.float32) #'Tensor' object does not support item assignment, cant build Ypre
    
    if flag == 1:
        rownum = batch_size
    elif flag == 2:
        rownum = num
    elif flag == 3:
        rownum = num
    
    for i in range(rownum):
        Xtem = x[i,:]

        Atem1 = 0.5*(weights['A1'] + tf.transpose(weights['A1']))#+ Atem 
        Atem1 = normalize_adj(Atem1)      
        Z1 = tf.matmul(Atem1, Xtem) 
        layer_1 = tf.add(tf.matmul(Z1, weights['h1']), biases['b1'])
        layer_1 = tf.nn.relu(layer_1)
        
        Atem2 = 0.5*(weights['A2'] + tf.transpose(weights['A2']))#+ Atem 
        Atem2 = normalize_adj(Atem2)
        Z2 = tf.matmul(Atem2, layer_1)
        layer_2 = tf.add(tf.matmul(Z2, weights['h2']), biases['b2'])
        layer_2 = tf.nn.relu(layer_2)
        
        Atem3 = 0.5*(weights['A3'] + tf.transpose(weights['A3']))#+ Atem 
        Atem3 = normalize_adj(Atem3)
        Z3 = tf.matmul(Atem3, layer_2)
        layer_3 = tf.add(tf.matmul(Z3, weights['h3']), biases['b3'])
        layer_3 = tf.nn.relu(layer_3)
        
        Z4 = layer_3
        out_layer = tf.add(tf.matmul(Z4, weights['out']), biases['bout'])

        if i ==0:
            #print (out_layer.shape)
            tem = tf.reshape(out_layer, [1, n_output_vec])
            output_list = tem
        else:
            tem = tf.reshape(out_layer, [1, n_output_vec])
            output_list = tf.concat([output_list, tem], 0)

    
    return output_list

In [94]:
def gcn_corr_final(feature_in, horizon, learning_rate, decay, batch_size, n_hidden_vec1, n_hidden_vec2, n_hidden_vec3, keep, early_stop_th, training_epochs):
   
    feature_in = int(feature_in) 
    
    early_stop_k = 0 # early stop patience
    display_step = 1 # frequency of printing results
    best_val = 10000
    traing_error = 0
    test_error = 0
    predic_res = []

    tf.reset_default_graph()

    batch_size = int(batch_size)
    n_hidden_vec1 = int(n_hidden_vec1)
    n_hidden_vec2 = int(n_hidden_vec2)
    n_hidden_vec3 = int(n_hidden_vec3)
    early_stop_th = int(early_stop_th)
    training_epochs = int(training_epochs)

    # tf Graph input
    X = tf.placeholder(tf.float32, [None, sn, feature_in]) # X is the input signal
    Y = tf.placeholder(tf.float32, [None, n_output_vec]) # y is the regression output
    
    keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

    # Store layers weight & bias
    weights = {
        'h1': tf.Variable(tf.random_normal([feature_in, n_hidden_vec1])),
        'h2': tf.Variable(tf.random_normal([n_hidden_vec1, n_hidden_vec2])),
        'h3': tf.Variable(tf.random_normal([n_hidden_vec2, n_hidden_vec3])),
        'out': tf.Variable(tf.random_normal([n_hidden_vec3, 1])),
        'A1': tf.Variable(tf.random_normal([sn, sn])),
        'A2': tf.Variable(tf.random_normal([sn, sn])),
        'A3': tf.Variable(tf.random_normal([sn, sn])),
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([sn, 1])),
        'b2': tf.Variable(tf.random_normal([sn, n_hidden_vec2])), 
        'b3': tf.Variable(tf.random_normal([sn, n_hidden_vec3])),
        'bout': tf.Variable(tf.random_normal([sn, 1])), 
    }

    # Construct model
    pred= gcn(X, weights, biases, batch_size, 1, n_output_vec)
    
    pred = scaler.inverse_transform(pred)
    Y_true_tr = scaler.inverse_transform(Y)
    cost = tf.reduce_mean(tf.pow(pred - Y_true_tr, 2)) 

    pred_val= gcn(X, weights, biases, batch_size, 2, n_output_vec, num_val)
    pred_val = scaler.inverse_transform(pred_val)
    Y_true_val = scaler.inverse_transform(Y)
    cost_val =  tf.reduce_mean(tf.pow(pred_val - Y_true_val, 2)) 

    pred_tes= gcn(X, weights, biases, batch_size, 3, n_output_vec, num_test)
    pred_tes = scaler.inverse_transform(pred_tes)
    Y_true_tes = scaler.inverse_transform(Y)
    cost_tes = tf.reduce_mean(tf.pow(pred_tes - Y_true_tes, 2)) 
                                         
    optimizer = tf.train.RMSPropOptimizer(learning_rate, decay).minimize(cost)
    #optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

    # Initializing the variables
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

    with tf.Session() as sess:
        sess.run(init)

        for epoch in range(training_epochs):

            avg_cost = 0.
            total_batch = int(num_train/batch_size)

            for i in range(total_batch):
                
                _, c = sess.run([optimizer, cost], feed_dict={X: X_training[i*batch_size:(i+1)*batch_size,], 
                                                      Y: Y_training[i*batch_size:(i+1)*batch_size,], 
                                                              keep_prob: keep})

                avg_cost += c / total_batch 
            
            avg_cost = np.sqrt(avg_cost)
             #Display logs per epoch step
            if epoch % display_step == 0:
                print ("Epoch:", '%04d' % (epoch+1), "Training RMSE=", \
                    "{:.9f}".format(avg_cost))
            # validation
            c_val = sess.run([cost_val], feed_dict={X: X_val, Y: Y_val,  keep_prob:1})
            c_val = np.sqrt(c_val[0])
            print("Validation RMSE: ", c_val)
            # testing
            c_tes, preds, A1, Y_true = sess.run([cost_tes, pred_tes, weights['A1'], Y_true_tes], feed_dict={X: X_test,Y: Y_test, keep_prob: 1})
            c_tes = np.sqrt(c_tes)

            if c_val < best_val:
                best_val = c_val
                # save model
                #saver.save(sess, './bikesharing_gcnn_ddgf')
                test_error = c_tes
                traing_error = avg_cost
                predic_res = preds
                early_stop_k = 0 # reset to 0

            # update early stopping patience
            if c_val >= best_val:
                early_stop_k += 1

            # threshold
            if early_stop_k == early_stop_th:
                break
            

        print("epoch is ", epoch)
        print("training RMSE is ", traing_error)
        print("Optimization Finished! the lowest validation RMSE is ", best_val)
        print("The test RMSE is ", test_error)
    
    #test_Y = Y_test
    #test_error = np.sqrt(test_error)
    return best_val, predic_res,Y_true,test_error, A1

# Import Data

In [95]:
file_Name = "data/NYCBikeHourly272.pickle"
fileObject = open(file_Name,'rb') 
hourly_bike = pickle.load(fileObject) 
hourly_bike = pd.DataFrame(hourly_bike)

# Split Data into Training, Validation and Testing

In [96]:
class StandardScaler:
    """
    Standard the input
    """

    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def transform(self, data):
        return (data - self.mean) / self.std

    def inverse_transform(self, data):
        return (data * self.std) + self.mean


In [97]:
sn = 272 # station number 
feature_in = 24 # number of features at each node, e.g., bike sharing demand from past 24 hours
horizon = 1 # the length to predict, e.g., predict the future one hour bike sharing demand

X_whole = []
Y_whole = []

x_offsets = np.sort(
    np.concatenate((np.arange(-feature_in+1, 1, 1),))
)

y_offsets = np.sort(np.arange(1, 1+ horizon, 1))

min_t = abs(min(x_offsets))
max_t = abs(hourly_bike.shape[0] - abs(max(y_offsets)))  # Exclusive
for t in range(min_t, max_t):
    x_t = hourly_bike.iloc[t + x_offsets, 0:sn].values.flatten('F')
    y_t = hourly_bike.iloc[t + y_offsets, 0:sn].values.flatten('F')
    X_whole.append(x_t)
    Y_whole.append(y_t)

X_whole = np.stack(X_whole, axis=0)
Y_whole = np.stack(Y_whole, axis=0)

n_input_vec = X_whole.shape[1] # e.g., 272 * 24
n_output_vec = Y_whole.shape[1] # each row represent a result


X_whole = np.reshape(X_whole, [X_whole.shape[0], sn, feature_in])
num_samples = X_whole.shape[0]
num_train = 20000 # Note here actually we use the first 20000 to train the model. The paper mentioned "22304" need to be corrected.
num_val = 2000
num_test = 2000

X_training = X_whole[:num_train, :]
Y_training = Y_whole[:num_train, :]

# shuffle the training dataset
perm = np.arange(X_training.shape[0])
np.random.shuffle(perm)
X_training = X_training[perm]
Y_training = Y_training[perm]

X_val = X_whole[num_train:num_train+num_val, :]
Y_val = Y_whole[num_train:num_train+num_val, :]

X_test = X_whole[num_train+num_val:num_train+num_val+num_test, :]
Y_test = Y_whole[num_train+num_val:num_train+num_val+num_test, :]



In [98]:
scaler = StandardScaler(mean=X_training.mean(), std=X_training.std())

X_training = scaler.transform(X_training)
Y_training = scaler.transform(Y_training)

X_val = scaler.transform(X_val)
Y_val = scaler.transform(Y_val)

X_test = scaler.transform(X_test)
Y_test = scaler.transform(Y_test)

# Hyperparameters

In [99]:
sn = 272 # station number 
feature_in = 24 # number of features at each node, e.g., bike sharing demand from past 24 hours
horizon = 1 # the length to predict, e.g., predict the future one hour bike sharing demand

learning_rate = 0.005 # learning rate
decay = 0.9
batchsize = 100 # batch size 

n_hidden_vec1 = 10 # feature length of the first hidden layer after the graph convolution
n_hidden_vec2 = 10 # feature length of the second ...
n_hidden_vec3 = 5 # feature length of the third ...

keep = 1 # drop out probability

early_stop_th = 20 # early stopping threshold, if validation RMSE not dropping in continuous 20 steps, break
training_epochs = 100 # total training epochs


# Training

In [100]:
a = datetime.datetime.now()

val_error, predic_res, test_Y, test_error, A1 = gcn_corr_final(feature_in, horizon, learning_rate, decay, batchsize, n_hidden_vec1,
                                                            n_hidden_vec2, n_hidden_vec3, keep, early_stop_th, training_epochs)


b = datetime.datetime.now()

print('Total training time: ', b-a)

#np.savetxt("prediction.csv", predic_res, delimiter = ',')
#np.savetxt("prediction_Y.csv", test_Y, delimiter = ',')


Epoch: 0001 Training RMSE= 45.114036799
Validation RMSE:  5.108541
Epoch: 0002 Training RMSE= 4.017966649
Validation RMSE:  3.7551003
Epoch: 0003 Training RMSE= 3.415668886
Validation RMSE:  3.5079467
Epoch: 0004 Training RMSE= 3.221278121
Validation RMSE:  3.4558823
Epoch: 0005 Training RMSE= 3.121586691
Validation RMSE:  3.4163816
Epoch: 0006 Training RMSE= 3.057729643
Validation RMSE:  3.3816373
Epoch: 0007 Training RMSE= 3.008908463
Validation RMSE:  3.3330045
Epoch: 0008 Training RMSE= 2.973043703
Validation RMSE:  3.3097322
Epoch: 0009 Training RMSE= 2.945620377
Validation RMSE:  3.2862923
Epoch: 0010 Training RMSE= 2.923042358
Validation RMSE:  3.2650893
Epoch: 0011 Training RMSE= 2.903385853
Validation RMSE:  3.2435055
Epoch: 0012 Training RMSE= 2.886005596
Validation RMSE:  3.2216015
Epoch: 0013 Training RMSE= 2.870490554
Validation RMSE:  3.202019
Epoch: 0014 Training RMSE= 2.857524086
Validation RMSE:  3.1865935
Epoch: 0015 Training RMSE= 2.845514433
Validation RMSE:  3.1726

In [ ]:
# use three layers: 2.34